# Exercise for PhD students and interested Master students

We want to learn whether European equity markets pay a premium for taking on systematic risks.


## Task (A): Reading-in Data

A.1 Load daily equity returns of Euro Stoxx 50 constituents, i.e. r_ES50_d_cleaned_realized.csv

A.2 Load daily Euro spot rates, i.e. r_ecb_2004_2020_d.csv

A.3 Load daily return data from the Fama French database (market, size, HML), i.e. FF5_d.csv


## Task (B): Preparing Data

B.1 Treat 3 month spot rate as the risk-free rate. Merge data into a pandas dataframe called data_eu_f. 

B.2 Notice, the read-in data has different scalings right now. Risk-free rate is in annualized %, equity returns are in daily decimals, Fama French factors are in daily %. Bring data_eu_f into daily decimals. Also, ensure to swap the market return from Fama and French with the 1/N portfolio return from r_ES50_d_cleaned_realized.csv. Notice, we swap the market return again with the excess market return, meaning you need to subtract the the risk-free rate. Hint: when going from annual to daily we assume 252 days convention (252 days = 1 year).

## Task (C): Fama-MacBeth Regressions

C.1 Write a function called GetBeta. It takes as input data_eu_f as well as the ticker names. The function returns factor loadings of each ticker. Loadings are computed with regard to the factors markets' excess return, SMB, HML, RMB and CMA.
Hint: Use GLS regressions
Hint: Use statsmodels package for GLS regressions
Hint: you regress excess returns onto the factors
Hint: Adidas has the following loadings: -0.24 (CMA), -0.098 (HML), 0.85 (MKT), 0.05 (RMW), 0.09 (SMB)


C.2 Write a function called GetLambda. The function takes as input data_eu_f and the loadings from C.1. The function runs cross-sectional regressions for each time point t and returns a matrix. The rows contains time periods whereas the columns contain the date-t realized market prices for CMA, HML, MKT, RMW and SMB.
Hint: use GLS regressions
Hint: the realized market prices of risk for the last time point are: -0.02 (CMA), -0.015 (HML), -0.0037 (MKT), -0.023 (RMW), 0.023 (SMB), 0.0020 (constant).
Hint: the constant is the alpha.

C.3 Fama and MacBeth recommend to compute the unconditional estimate of the market price of risk as the sample mean. Do so, then multiply the solution by 25200 to have the estimate in annual % units.
Hint: The solution is 
CMA     -11.884396 %
HML     -14.551905 %
MER      -3.242965 %
RMW       1.850954 %
SMB       2.353284 %
alpha     9.590819 %

C.4 Compute the asymptotic t-statistic for the unconditional market prices of risk from C.3. Is there evidence for a non-zero market price of risk?
Hint: 

In [1]:
#packages
import pandas as pd
import numpy as np
import statsmodels.api as sm

## Solution A.1

In [2]:
#daily euro-area stock returns
r_d = pd.read_csv('r_ES50_d_cleaned_realized.csv', parse_dates = True, index_col=0)

## Solution A.2

In [3]:
#euro rf
y_ecb = pd.read_csv('y_ecb_2004_2020_d.csv', parse_dates = True, index_col=0)


## Solution A.3


In [4]:
# US factor returns
f_d = pd.read_csv('FF5_d.csv', parse_dates=True, index_col=0)


## Solution B.1


In [5]:
#Merge data to same timeline
data_eu = pd.concat([y_ecb["0.25 Y"], r_d], axis=1, sort=False)
data_eu_f = pd.concat([data_eu, f_d], axis=1, sort=False)
data_eu_f = data_eu_f.dropna()


## Solution B.2

In [6]:
#data prep
##Note: 0.25 Y is annualized %
##Note: r_d is daily decimals
##Note: f_d is daily %

##Ergo: Have all returns as daily decimals

data_eu_f["0.25 Y"] = data_eu_f["0.25 Y"] / (252*100)
data_eu_f = data_eu_f.rename(columns={'1/N': 'MER'})
data_eu_f['MER'] = data_eu_f['MER'] - data_eu_f['0.25 Y']
data_eu_f = data_eu_f.drop(['Mkt-RF', 'RF' ], axis=1)
data_eu_f["SMB"] = data_eu_f["SMB"] / 100
data_eu_f["HML"] = data_eu_f["HML"] / 100
data_eu_f["RMW"] = data_eu_f["RMW"] / 100
data_eu_f["CMA"] = data_eu_f["CMA"] / 100

#just in case
data_eu_f = data_eu_f.dropna()

## Solution C.1






In [7]:
#TickerNames of r_d
beta_i = data_eu_f.agg('mean')
beta_i = beta_i.drop(['0.25 Y', 'MER', 'SMB', 'HML', 'RMW', 'CMA'], axis=0)
TickerNames = beta_i.index

In [8]:
def GetBeta(data_eu_f, TickerNames): 
         
    betas_i = pd.DataFrame()
    
    X = data_eu_f.iloc[:,-5:]
    X = sm.add_constant(X)
    for i in range(1,44):
        #set y
        y = data_eu_f.iloc[:,i] - data_eu_f["0.25 Y"]
        
        #GLS
        linReg = sm.GLS(y,X)
        output = linReg.fit()
        output_beta_i = output.params
        
        betas_i = betas_i.append(output_beta_i, ignore_index=True)
    
    betas_i = betas_i.set_index(TickerNames)
    betas_i = betas_i.drop(['const'], axis=1)
    
    return (betas_i)
        

In [9]:
beta_i = GetBeta(data_eu_f, TickerNames)
print(beta_i)

               CMA       HML       MER       RMW       SMB
ADS.DE   -0.243772 -0.097618  0.851697  0.050128  0.090717
AI.PA     0.045496 -0.143370  0.855699 -0.024617 -0.000343
ALV.DE   -0.253101  0.160126  1.178462 -0.129511 -0.096547
ASML.AS  -0.235243 -0.227791  0.889438 -0.287252  0.179596
CS.PA    -0.085029  0.163856  1.457995 -0.241171 -0.099821
BBVA.MC  -0.041478  0.302653  1.262021  0.060185  0.029287
SAN.MC    0.079578  0.277532  1.296727  0.036910  0.024292
BAS.DE   -0.024021  0.046740  1.062733  0.107708  0.123994
BAYN.DE   0.160103 -0.235375  0.950234 -0.057504  0.053269
BMW.DE   -0.127041  0.069487  1.101876  0.085578  0.143806
BNP.PA   -0.150860  0.416631  1.348611 -0.197138  0.002106
CRG.IR    0.116451  0.078503  1.047814  0.034205  0.167344
SGO.PA   -0.001828 -0.042827  1.355478  0.081573  0.140385
DAI.DE   -0.155665  0.125811  1.290761  0.305240  0.266105
DTE.DE    0.172633 -0.153924  0.785542 -0.097720 -0.060086
ENEL.MI   0.241394 -0.133576  0.964940 -0.007218 -0.1809

## Solution C.2

In [10]:
def GetLambda_t(data_eu_f, beta_i):
    
    X = beta_i
    X = sm.add_constant(X)
    
    Lambda = pd.DataFrame()
    for t in range(0,data_eu_f.shape[0]):
        y = data_eu_f.iloc[t,1:44] - data_eu_f.iloc[t,0]
        
        #GLS
        linReg = sm.GLS(y,X)
        output = linReg.fit()
        lambda_t = output.params
        Lambda = Lambda.append(lambda_t, ignore_index=True)
 
    return Lambda

In [11]:
Lambda = GetLambda_t(data_eu_f, beta_i)
Lambda.tail(2)

,CMA,HML,MER,RMW,SMB,const
3935,0.020219,0.023885,0.031956,0.019575,-0.006056,-0.019318
3936,-0.020558,-0.015129,-0.003699,-0.023706,0.023710,0.001970


In [12]:
Lambda = Lambda.rename(columns={'const': 'alpha'})

## Solution C.3

In [13]:
MPR_uncond = Lambda.mean() * 252 * 100 #annualized in %
print(MPR_uncond)

CMA     -11.884396
HML     -14.551905
MER      -3.242965
RMW       1.850954
SMB       2.353284
alpha     9.590819
dtype: float64


## Solution C.4

In [14]:
MPR_t = Lambda.mean() / (Lambda.std() / np.sqrt(data_eu_f.shape[0]))
print(MPR_t)

CMA     -1.633429
HML     -1.375347
MER     -0.365656
RMW      0.236060
SMB      0.207018
alpha    1.361660
dtype: float64


**Interpretation:**

All t-stats are far away from 3. Statistically, there is strong evidence that the Fama-French risk factors were not priced in European equity at the daily frequency.